In [ ]:
# Imports
import os
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Custom Dataset class to handle input encodings and labels
class HarassmentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# Function to compute metrics
def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [ ]:
def main(path, checkpoint=False):
  # Load the dataset
    data_path = path+"/balanced_binary_classification_dataset.csv"
    df = pd.read_csv(data_path)

    texts = df['text'].tolist()
    labels = df['label'].tolist()

    # Split data in training,evaluation and testing with a percentage of 70,15,15
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        texts, labels, test_size=0.3, random_state=42
    )
    test_texts, val_texts, test_texts, val_labels = train_test_split(
        val_texts, val_labels, test_size=0.5, random_state=42
    )

    # Load the model and its tokenizer
    model_name = "microsoft/deberta-v3-small"
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Tokenize the training and validation sets
    train_dataset = HarassmentDataset(train_texts, train_labels, tokenizer)
    val_dataset = HarassmentDataset(val_texts, val_labels, tokenizer)

    # Definition of the parameters for the training
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    training_args = TrainingArguments(
        output_dir=path+"/results",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=1e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=5,
        weight_decay=0.01,
        logging_dir=path+"/logs",
        logging_steps=10,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        save_total_limit=2,
    )

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    # If we are resuming from a checkpoint we load it, otherwise we start the training
    if checkpoint:
        trainer.train(resume_from_checkpoint=True)
    else:
        trainer.train()
    # At the end of the training we save the best model
    model.save_pretrained(path+"/model_save")
    tokenizer.save_pretrained(path+"/model_save")

    metrics = trainer.evaluate()
    print("Evaluation Metrics:", metrics)

In [ ]:
# Entry point for script
if __name__ == "__main__":
    main(path="/content/drive/My Drive/Projet IA",checkpoint=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.317500,0.280608,0.885018,0.848946,0.929420,0.887362
2,0.228700,0.287220,0.892587,0.869292,0.917542,0.892766
3,0.262900,0.340581,0.893255,0.860882,0.931476,0.894789
4,0.141000,0.380236,0.895703,0.869761,0.924395,0.896246
5,0.199800,0.417202,0.896037,0.869369,0.925765,0.896681


Evaluation Metrics: {'eval_loss': 0.4172024726867676, 'eval_accuracy': 0.8960373998219056, 'eval_precision': 0.8693693693693694, 'eval_recall': 0.9257651895842851, 'eval_f1': 0.8966814159292036, 'eval_runtime': 45.0199, 'eval_samples_per_second': 199.556, 'eval_steps_per_second': 12.483, 'epoch': 5.0}
